In [1]:
import requests
import json
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', None)

In [2]:
!Pwd

/Users/a_conte/Documents/Flatiron/Phase_1_Project


In [3]:
!ls -a

.                                Business_Details_Questions.ipynb
..                               Call_Business_Details.ipynb
.Bostonbar.csv                   Call_Reviews.ipynb
.DS_Store                        Data
.NewYorkBars.csv                 Images
.git                             Phase_1_Project_Charts.ipynb
.gitignore                       Review_Helper.py
.ipynb_checkpoints               Script_Use.py
.secrets                         Untitled.ipynb
Boston_Bars_Clean.csv            __pycache__
Boston_Bars_Details.csv


In [4]:
!echo '{"id": "qD3df2spUAnavWJQ5NNDJA", "key": "lDOH3Sz2cLDQi3QI4dAx5iXC6QHj8OIdGLzmNiPDFaHSo6GQJjlN9yfXG4WE_kBzRJ79DU56XmUsP63hgIMWKaC9ryfwb_yXgCIkVq3PKIRBTRpH6-a3AUTtOEqZYHYx"}' > creds.json

In [5]:
!ls -a

.                                Business_Details_Questions.ipynb
..                               Call_Business_Details.ipynb
.Bostonbar.csv                   Call_Reviews.ipynb
.DS_Store                        Data
.NewYorkBars.csv                 Images
.git                             Phase_1_Project_Charts.ipynb
.gitignore                       Review_Helper.py
.ipynb_checkpoints               Script_Use.py
.secrets                         Untitled.ipynb
Boston_Bars_Clean.csv            __pycache__
Boston_Bars_Details.csv          creds.json


In [6]:
!mv creds.json .secrets/

In [7]:
!ls -a .secrets/

.          ..         creds.json


In [8]:
with open(".secrets/creds.json") as f:
    creds = json.load(f)

In [9]:
url = 'https://api.yelp.com/v3/businesses/search'
term = 'sports bar'
location = 'Boston'
#SEARCH_LIMIT = 50
headers = {
    'Authorization': 'Bearer ' + creds['key']
}

url_params = {
                "term": term.replace(' ', '+'),
                "location": location.replace(' ', '+'),
                "limit": 50,
                "offset": 0,
    
    
    
            }

response = requests.get(url, headers=headers, params=url_params)
print(response.status_code)

200


In [10]:
from Script_Use import *

In [11]:
csv_filepath = (r'/Users/a_conte/Documents/Flatiron/Phase_1_Project/Data/Boston_Bars_Details.csv')

In [12]:
#since the yelp API limits the amount of results that can be returned from your search, we need to run it through a while loop to basically extract information from the next page of data. (Whatever our curator is set to, you can only grab 50 results at a time so we set the curator to the number of results we want to grab. )

cur = 0
#set up a while loop to go through and grab the result 
while cur < 300:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number.  
    #We're saving the result of the yelp call and then turning that result into a .json file so that we can extract businesses from the dictionary that is now a .json file. 
    #the yelp api basically returns a string of information, then when we call .json on that result, we're turning it into a dictionary so that we can parse through the information.  Parse just means filter. 
    #From that dictionary, we're extracting the businesses and we're going to use that information to save to our .csv file.  That's where the clean and everything else comes in.  
    #A tuple can exist outside of the dataframe, it's just any set of data within a parentheses.  They can't be mutated or overwritten. Once you assign a tuple, you can't change it.  If you made a tuple in the beginning of your code, you can't change the tuple but you can alter the variables that live within the tuple.  
    results = yelp_call(url_params, creds)
    yelp_results = results.json()
    business = yelp_results['businesses']
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_data(business)
    # use your function to insert your parsed results into the db
    
    df_save(csv_filepath ,parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 50

In [13]:
df = pd.read_csv(csv_filepath)

In [14]:
df

,0,The Corner Pub,"['162 Lincoln St', 'Boston, MA 02111']",tCPxYdZVbsDooH3DQsNLqQ,4.5,134,"{'latitude': 42.35076, 'longitude': -71.05837}",$
0,1,Bleacher Bar,"['82 Lansdowne St', 'Fenway Park', 'Boston, MA 02228']",47IRTOlC4kjAf1aFKLZDdA,4.0,483,"{'latitude': 42.3469855442944, 'longitude': -71.096686816461}",$$
1,2,21st Amendment,"['150 Bowdoin Sq', 'Boston, MA 02108']",ZrSNKHiKK_PBB6UHx22FaA,4.0,401,"{'latitude': 42.358355, 'longitude': -71.062686}",$$
2,3,Boston Brew Tours by City Brew Tours,"['2 Park Plz', 'Boston, MA 02116']",UMtKPjI5xsR5Db0pCyECFA,4.5,169,"{'latitude': 42.3522696, 'longitude': -71.0670277}",$$
3,4,Bostonia Public House,"['131 State St', 'Boston, MA 02109']",xZ22_0eJOgaiKqIhoSHrPw,4.0,1455,"{'latitude': 42.359213, 'longitude': -71.054296}",$$
4,5,The Gallows,"['1395 Washington St', 'Boston, MA 02118']",5HMXgD_gui5n0Tc_hadesg,4.0,857,"{'latitude': 42.341371, 'longitude': -71.070237}",$$
...,...,...,...,...,...,...,...,...
1194,45,Galway House,"['710 Centre St', 'Jamaica Plain, MA 02130']",oSM8ItF9hZ0Qz7-y9ckuiQ,4.0,233,"{'latitude': 42.311316, 'longitude': -71.114221}",$$
1195,46,Metropolis,"['584 Tremont St', 'Boston, MA 02118']",geqpfK2WO6uHJCDpNofURA,4.0,493,"{'latitude': 42.34324, 'longitude': -71.072621}",$$
1196,47,Burro Bar Brookline,"['1665 Beacon St', 'Brookline, MA 02445']",LgKBFgM0hN2yyX9WfEF_JQ,4.0,218,"{'latitude': 42.33849, 'longitude': -71.13739}",$$
1197,48,TGI Fridays,"['1626 Tremont St', 'Boston, MA 02120']",6uuWg3RYUYzrLePphT3PQw,2.0,228,"{'latitude': 42.3336822, 'longitude': -71.1038111}",$$


In [15]:
df = df_clean(csv_filepath)

In [16]:
df

,Name,Address,Business ID,Rating,Review Count,Coordinates,Price
0,The Corner Pub,"['162 Lincoln St', 'Boston, MA 02111']",tCPxYdZVbsDooH3DQsNLqQ,4.5,134,"{'latitude': 42.35076, 'longitude': -71.05837}",1
1,Bleacher Bar,"['82 Lansdowne St', 'Fenway Park', 'Boston, MA 02228']",47IRTOlC4kjAf1aFKLZDdA,4.0,483,"{'latitude': 42.3469855442944, 'longitude': -71.096686816461}",2
2,21st Amendment,"['150 Bowdoin Sq', 'Boston, MA 02108']",ZrSNKHiKK_PBB6UHx22FaA,4.0,401,"{'latitude': 42.358355, 'longitude': -71.062686}",2
3,Boston Brew Tours by City Brew Tours,"['2 Park Plz', 'Boston, MA 02116']",UMtKPjI5xsR5Db0pCyECFA,4.5,169,"{'latitude': 42.3522696, 'longitude': -71.0670277}",2
4,Bostonia Public House,"['131 State St', 'Boston, MA 02109']",xZ22_0eJOgaiKqIhoSHrPw,4.0,1455,"{'latitude': 42.359213, 'longitude': -71.054296}",2
...,...,...,...,...,...,...,...
1151,Galway House,"['710 Centre St', 'Jamaica Plain, MA 02130']",oSM8ItF9hZ0Qz7-y9ckuiQ,4.0,233,"{'latitude': 42.311316, 'longitude': -71.114221}",2
1152,Metropolis,"['584 Tremont St', 'Boston, MA 02118']",geqpfK2WO6uHJCDpNofURA,4.0,493,"{'latitude': 42.34324, 'longitude': -71.072621}",2
1153,Burro Bar Brookline,"['1665 Beacon St', 'Brookline, MA 02445']",LgKBFgM0hN2yyX9WfEF_JQ,4.0,218,"{'latitude': 42.33849, 'longitude': -71.13739}",2
1154,TGI Fridays,"['1626 Tremont St', 'Boston, MA 02120']",6uuWg3RYUYzrLePphT3PQw,2.0,228,"{'latitude': 42.3336822, 'longitude': -71.1038111}",2


In [17]:
df.to_csv(r'/Users/a_conte/Documents/Flatiron/Phase_1_Project/Data/Boston_Bars_Clean_Use).csv')

In [18]:
df_NY_Bars = pd.read_csv((r'/Users/a_conte/Documents/Flatiron/Phase_1_Project/Data/NYC_Bars_Clean.csv'), index_col = 0)
df_Boston_Bars = pd.read_csv((r'/Users/a_conte/Documents/Flatiron/Phase_1_Project/Data/Boston_Bars_Clean.csv'), index_col = 0)
df_NY_Reviews = pd.read_csv((r'/Users/a_conte/Documents/Flatiron/Phase_1_Project/Data/NYC_Reviews_Clean.csv'), index_col = 0)
df_Boston_Reviews = pd.read_csv((r'/Users/a_conte/Documents/Flatiron/Phase_1_Project/Data/Boston_Reviews_Clean.csv'), index_col = 0)

In [19]:
df_NY_Bars.describe()

,Rating,Review Count,Price
count,291.000000,291.000000,291.000000
mean,3.943299,442.213058,1.872852
std,0.361102,525.919381,0.506154
min,3.000000,12.000000,1.000000
25%,4.000000,148.000000,2.000000
50%,4.000000,266.000000,2.000000
75%,4.000000,526.000000,2.000000
max,5.000000,4397.000000,4.000000


In [20]:
df_NY_Reviews.describe()

,Rating
count,873.000000
mean,4.352806
std,1.008755
min,1.000000
25%,4.000000
50%,5.000000
75%,5.000000
max,5.000000


In [21]:
df_Boston_Bars.describe()

,Rating,Review Count,Price
count,289.000000,289.000000,289.000000
mean,3.724913,493.179931,2.010381
std,0.483695,534.594578,0.523639
min,2.000000,6.000000,1.000000
25%,3.500000,201.000000,2.000000
50%,4.000000,348.000000,2.000000
75%,4.000000,601.000000,2.000000
max,5.000000,5165.000000,4.000000


In [22]:
df_Boston_Reviews.describe()

,Rating
count,867.000000
mean,4.377163
std,0.976436
min,1.000000
25%,4.000000
50%,5.000000
75%,5.000000
max,5.000000


In [23]:
# Which are the 5 most reviewed businesses in your dataset?

In [24]:
#New York
df_NY_Bars.nlargest(5,['Review Count'])

,Name,Address,Business ID,Rating,Review Count,Coordinates,Price
159,Jacob's Pickles,"['509 Amsterdam Ave', 'New York, NY 10024']",xt4sa64WOrpJvZBDPNPNYg,4.0,4397,"{'latitude': 40.7866504411994, 'longitude': -73.9755284786224}",2
270,Jane,"['100 W Houston St', 'New York, NY 10012']",uoT2_Wrt1noD6kZOID7tUg,4.0,3149,"{'latitude': 40.7273779, 'longitude': -74.00025}",2
68,Dinosaur Bar-B-Que,"['700 W 125th St', 'New York, NY 10027']",XzUW9MvEGOFs9_aNj11j2Q,4.0,3120,"{'latitude': 40.818098338924, 'longitude': -73.9608729269837}",2
166,Gramercy Tavern,"['42 E 20th St', 'New York, NY 10003']",veq1Bl1DW3UWMekZJUsG1Q,4.5,2941,"{'latitude': 40.73844, 'longitude': -73.98825}",4
170,Juliana's,"['19 Old Fulton St', 'Brooklyn, NY 11201']",ysqgdbSrezXgVwER2kQWKA,4.5,2295,"{'latitude': 40.70274718768062, 'longitude': -73.99343490196397}",2


In [25]:
#Boston
df_Boston_Bars.nlargest(5,['Review Count'])

,Name,Address,Business ID,Rating,Review Count,Coordinates,Price
13,Neptune Oyster,"['63 Salem St', 'Boston, MA 02113']",y2w6rFaO0XEiG5mFfOsiFA,4.5,5165,"{'latitude': 42.36321, 'longitude': -71.05594}",3
144,Union Oyster House,"['41 Union St', 'Boston, MA 02108']",Ruq91rmQNweZ78TzKylQMg,3.5,2855,"{'latitude': 42.36126, 'longitude': -71.05688}",2
197,Atlantic Fish Co,"['761 Boylston St', 'Boston, MA 02116']",AGR_kBvN__I7aTl0rBywAg,4.0,2604,"{'latitude': 42.3492217873536, 'longitude': -71.0811272917217}",3
43,The Beehive,"['541 Tremont St', 'Boston, MA 02116']",367aDatN4B0UNT4hNPMqPQ,3.5,2195,"{'latitude': 42.344493, 'longitude': -71.071497}",2
19,Lolita Cocina & Tequila Bar Back Bay,"['271 Dartmouth St', 'Boston, MA 02116']",u51e3gychuqBYNqe2xG6_w,4.0,2176,"{'latitude': 42.3505860446405, 'longitude': -71.077508474716}",2


In [26]:
# What is the highest rating recieved in your data set and how many businesses have that rating?

In [27]:
#New York
df_NY_Bars.nlargest(1, ['Rating'])

,Name,Address,Business ID,Rating,Review Count,Coordinates,Price
15,The Commissioner,"['247 5th Ave', 'Brooklyn, NY 11215']",xHgMfdyLA5SPQz4c3ItTgg,5.0,141,"{'latitude': 40.67487, 'longitude': -73.98137}",1


In [28]:
#Boston
df_NY_Bars.nlargest(1, ['Rating'])

,Name,Address,Business ID,Rating,Review Count,Coordinates,Price
15,The Commissioner,"['247 5th Ave', 'Brooklyn, NY 11215']",xHgMfdyLA5SPQz4c3ItTgg,5.0,141,"{'latitude': 40.67487, 'longitude': -73.98137}",1


In [29]:
# What percentage of businesses have a rating greater than or 4.5?

In [30]:
#New York
NY_FourHalfStars = (df_NY_Bars['Rating'].values ==4.5).sum()
NY_FiveStars = (df_NY_Bars['Rating'].values == 5).sum()

#Boston
Boston_FourHalfStars = (df_Boston_Bars['Rating'].values ==4.5).sum()
Boston_FiveStars = (df_Boston_Bars['Rating'].values == 5).sum()

y = (Boston_FiveStars+Boston_FourHalfStars+NY_FiveStars+NY_FourHalfStars )
y_percent = ((y/580) * 100).round(2)
y_percent

13.62

In [31]:
# What percentage of businesses have a rating less than 3?

In [32]:
NY_Less = (df_NY_Bars['Rating'].values <= 3).sum()
Boston_Less = (df_Boston_Bars['Rating'].values <= 3).sum()
LessThan3 = (((NY_Less+Boston_Less)/580) *100).round()
LessThan3

9.0

In [33]:
# What percentage of your businesseshave a price label of one dollar sign? Two dollar signs? Three dollar signs? No dollar signs?

#One dollar sign
NY_Price1 = (df_NY_Bars['Price'].values == 1).sum()
Boston_Price1 = (df_Boston_Bars['Price'].values == 1).sum()
dataPrice1 = (((NY_Price1+Boston_Price1)/580)*100).round()
dataPrice1 

15.0

In [34]:
# Two dollar signs
NY_Price2 = (df_NY_Bars['Price'].values == 2).sum()
Boston_Price2 = (df_Boston_Bars['Price'].values == 2).sum()
dataPrice2 = (((NY_Price2+Boston_Price2)/580)*100).round()
dataPrice2 

76.0

In [35]:
# Three dollar signs
NY_Price3 = (df_NY_Bars['Price'].values == 3).sum()
Boston_Price3 = (df_Boston_Bars['Price'].values == 3).sum()
dataPrice3 = (((NY_Price3+Boston_Price3)/580)*100).round()
dataPrice3 

7.0

In [36]:
#No dollar signs
#Businesses with no dollar signs were extracted from our data set. 

In [37]:
#Return the text of the reviews for the most reviewed business.

In [38]:
# NY_Most_Reviewed   = df_NY_Bars.nlargest(1, ['Review Count'])
# Boston_Most_Reviewed = df_Boston_Bars.nlargest(1, ['Review Count'])

# NY_Most_Reviewed_Pull = df_NY_Bars.iloc[12]['Business ID']
# Boston_Most_Reviewed_Pull = df_Boston_Bars.iloc[12]['Business ID']

# NYMR = df_NY_Bars.iloc[12]['Business ID']
# BMR = df_Boston_Bars.iloc[12]['Business ID']

In [39]:
NY_Most_Reviewed   = df_NY_Bars.nlargest(1, ['Review Count'])
Boston_Most_Reviewed = df_Boston_Bars.nlargest(1, ['Review Count'])
NY_Most_Reviewed_Pull = df_NY_Bars.iloc[12]['Business ID']
Boston_Most_Reviewed_Pull = df_Boston_Bars.iloc[12]['Business ID']

In [40]:
NYMR = df_NY_Bars.iloc[12]['Business ID']
BMR = df_Boston_Bars.iloc[12]['Business ID']

In [41]:
# NY_Most_Reviewed   = df_NY_Bars.nlargest(1, ['Review Count'])
# Boston_Most_Reviewed = df_Boston_Bars.nlargest(1, ['Review Count'])

# NY_Most_Reviewed_Pull = df_NY_Bars.iloc[12]['Business ID']
# Boston_Most_Reviewed_Pull = df_Boston_Bars.iloc[12]['Business ID']

# NYMR = df_NY_Bars.iloc[12]['Business ID']
# BMR = df_Boston_Bars.iloc[12]['Business ID']

In [42]:
#New York
NY_Most_Reviewed_Text = df_NY_Reviews.loc[df_NY_Reviews['Business ID'] ==NYMR, ['Review']]

In [43]:
NY_Most_Reviewed_Text

,Review
36,"A truly special sports bar in the middle of East Village.\n\nYes, as others have explained, it's small. If you're more into the Midtown scene where you can..."
37,"What a gem of a bar in NYC. Came in here on Saturday for round 2 of the MLB playoffs. Upon arrival, they had a bouncer working the door who was a little..."
38,"Best find of 2018.\nAs of last year, I found my true sport spirit animal -- Hockey. It filled the void in my heart that I didn't even know exist.\n\nSo when..."


In [44]:


Boston_Most_Reviewed_Text = df_Boston_Reviews.loc[df_Boston_Reviews['Business ID'] ==BMR, ['Review']]

In [45]:
#Boston
Boston_Most_Reviewed_Text = df_Boston_Reviews.loc[df_Boston_Reviews['Business ID'] ==BMR, ['Review']]

In [46]:
# df_NY_Bars = pd.read_csv((r'/Users/a_conte/Documents/Flatiron/Phase_1_Project/Data/NYC_Bars_Clean.csv'), index_col = 0)
# df_Boston_Bars = pd.read_csv((r'/Users/a_conte/Documents/Flatiron/Phase_1_Project/Data/Boston_Bars_Clean.csv'), index_col = 0)
# df_NY_Reviews = pd.read_csv((r'/Users/a_conte/Documents/Flatiron/Phase_1_Project/Data/NYC_Reviews_Clean.csv'), index_col = 0)
# df_Boston_Reviews 



#Extracts all the code for biz at index loc

#New York
NY_Most_Reviewed_Text = df_NY_Reviews.loc[df_NY_Reviews['Business ID'] ==NYMR, ['Review']]

#Boston
Boston_Most_Reviewed_Text = df_Boston_Reviews.loc[df_Boston_Reviews['Business ID'] ==BMR, ['Review']]


In [47]:
NY_Most_Reviewed_Text

,Review
36,"A truly special sports bar in the middle of East Village.\n\nYes, as others have explained, it's small. If you're more into the Midtown scene where you can..."
37,"What a gem of a bar in NYC. Came in here on Saturday for round 2 of the MLB playoffs. Upon arrival, they had a bouncer working the door who was a little..."
38,"Best find of 2018.\nAs of last year, I found my true sport spirit animal -- Hockey. It filled the void in my heart that I didn't even know exist.\n\nSo when..."


In [48]:
Boston_Most_Reviewed_Text

,Review
36,My bf and I went to Boston for the weekend from Miami. We ended up at The Corner Tavern and it was great. Danielle ( I think that was her name) was very...
37,"Such a friendly place and the food is as tasty as ever. Love the Buffalo chicken wrap, cobb salad and grilled cheese....and of course the tots. Thanks to..."
38,The best neighborhood bar in Back Bay. Definitely worth stopping in for a bite or a beverage if you are in town. \n\nThe menu features creative takes on bar...


In [49]:
# Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the business with the most reviews.
# Find the lowest rated business and return text of the most recent review. If multiple business have the same rating, select the business with the least reviews.

In [50]:
NY_Highest_Rating_First_Review = df_NY_Bars.groupby(['Rating','Review Count'])
BOS_Highest_Rating_First_Review = df_Boston_Bars.groupby(['Rating','Review Count'])

In [51]:
#New York
NY_Highest_Rating_First_Review.first()

Name  \
Rating Review Count                                 
3.0    79            Bleachers Sports Bar & Grill   
       86                  The Bowery Beer Garden   
       96                            The Hideaway   
       148                       Desmond's Tavern   
       150                     Hudson Yards Grill   
...                                           ...   
4.5    1791                         Tanner Smiths   
       2295                             Juliana's   
       2941                       Gramercy Tavern   
5.0    141                       The Commissioner   
       928                             The Alcove   

                                                               Address  \
Rating Review Count                                                      
3.0    79                 ['240 Flatbush Ave', 'Park Slope, NY 11217']   
       86                          ['93 Bowery', 'New York, NY 10002']   
       96                       ['185 Duane St', 'New York, NY 10013']   
       148                    ['433 Park Ave S', 'New York, NY 10016']   
       150           ['20 Hudson Yards', 'Fl 4', 'New York, NY 10001']   
...                                                                ...   
4.5    1791                    ['204 W 55th St', 'New York, NY 10019']   
       2295                 ['19 Old Fulton St', 'Brooklyn, NY 11201']   
       2941                     ['42 E 20th St', 'New York, NY 10003']   
5.0    141                       ['247 5th Ave', 'Brooklyn, NY 11215']   
       928                    ['41-11 49th St', 'Sunnyside, NY 11104']   

                                Business ID  \
Rating Review Count                           
3.0    79            iLkU9Wo0Zte4DechmGnMqg   
       86            7yJ3fOQp3oj2xuDaUGD3_A   
       96            8ERDirrcagy6XNfVmeJt-g   
       148           CF4-nuTzsuBTg1Hxc2AcMg   
       150           esqICCGgA28y8Cg-l6mPZQ   
...                                     ...   
4.5    1791          z5hRX3iJ5Ty_S38iG_WY3Q   
       2295          ysqgdbSrezXgVwER2kQWKA   
       2941          veq1Bl1DW3UWMekZJUsG1Q   
5.0    141           xHgMfdyLA5SPQz4c3ItTgg   
       928           MNNpsRXe7Wt5m3Jla9Wk0w   

                                                                          Coordinates  \
Rating Review Count                                                                     
3.0    79              {'latitude': 40.6802563732559, 'longitude': -73.9749691478851}   
       86                               {'latitude': 40.7171, 'longitude': -73.99533}   
       96                            {'latitude': 40.717575, 'longitude': -74.010276}   
       148                             {'latitude': 40.74414, 'longitude': -73.98307}   
       150                         {'latitude': 40.7533572, 'longitude': -74.0002527}   
...                                                                               ...   
4.5    1791                            {'latitude': 40.76442, 'longitude': -73.98169}   
       2295          {'latitude': 40.70274718768062, 'longitude': -73.99343490196397}   
       2941                            {'latitude': 40.73844, 'longitude': -73.98825}   
5.0    141                             {'latitude': 40.67487, 'longitude': -73.98137}   
       928                          {'latitude': 40.7459065, 'longitude': -73.915103}   

                     Price  
Rating Review Count         
3.0    79                2  
       86                2  
       96                2  
       148               1  
       150               3  
...                    ...  
4.5    1791              2  
       2295              2  
       2941              4  
5.0    141               1  
       928               2  

[270 rows x 5 columns]

In [52]:
#Boston
BOS_Highest_Rating_First_Review.first()

Name  \
Rating Review Count                          
2.0    228                     TGI Fridays   
2.5    99            Longwood Grille & Bar   
       196                   World of Beer   
       275                The Greatest Bar   
       280                        The Harp   
...                                    ...   
4.5    1440           Samuel Adams Brewery   
       1520                Harpoon Brewery   
       5165                 Neptune Oyster   
5.0    6                 Allston Billiards   
       103             GrandTen Distilling   

                                                              Address  \
Rating Review Count                                                     
2.0    228                    ['1626 Tremont St', 'Boston, MA 02120']   
2.5    99                    ['342 Longwood Ave', 'Boston, MA 02115']   
       196            ['100 Cambridgeside Pl', 'Cambridge, MA 02141']   
       275                      ['262 Friend St', 'Boston, MA 02114']   
       280                     ['85 Causeway St', 'Boston, MA 02114']   
...                                                               ...   
4.5    1440                    ['30 Germania St', 'Boston, MA 02130']   
       1520                  ['306 Northern Ave', 'Boston, MA 02210']   
       5165                       ['63 Salem St', 'Boston, MA 02113']   
5.0    6             ['445 Cambridge St', 'Fl 2', 'Boston, MA 02134']   
       103                 ['383 Dorchester Ave', 'Boston, MA 02127']   

                                Business ID  \
Rating Review Count                           
2.0    228           6uuWg3RYUYzrLePphT3PQw   
2.5    99            GXI8J6Xi4e1A1nMPLNtkUA   
       196           uog_FdVRIPtReDYuxLdNGw   
       275           mkCTMbS4E-g2aC1oUTAuig   
       280           MjopvJwjCbOtsLvkUxUZNQ   
...                                     ...   
4.5    1440          VImbIWfxODVsiRHebSQePw   
       1520          FYDP6p5G9ZcI4UW7inh88A   
       5165          y2w6rFaO0XEiG5mFfOsiFA   
5.0    6             WhOZfqo49chqzdt15iluQA   
       103           KeC3GrFHGRl-vRT8LIuB3g   

                                                                        Coordinates  \
Rating Review Count                                                                   
2.0    228                       {'latitude': 42.3336822, 'longitude': -71.1038111}   
2.5    99                        {'latitude': 42.3385811, 'longitude': -71.1069107}   
       196                         {'latitude': 42.368131, 'longitude': -71.076239}   
       275            {'latitude': 42.364559173584, 'longitude': -71.0614166259766}   
       280                     {'latitude': 42.36451753, 'longitude': -71.06231418}   
...                                                                             ...   
4.5    1440                         {'latitude': 42.3144647, 'longitude': -71.1031}   
       1520          {'latitude': 42.3470249808051, 'longitude': -71.0348596407489}   
       5165                          {'latitude': 42.36321, 'longitude': -71.05594}   
5.0    6                             {'latitude': 42.35467, 'longitude': -71.13542}   
       103           {'latitude': 42.3350791931152, 'longitude': -71.0572052001953}   

                     Price  
Rating Review Count         
2.0    228               2  
2.5    99                2  
       196               2  
       275               2  
       280               2  
...                    ...  
4.5    1440              1  
       1520              1  
       5165              3  
5.0    6                 1  
       103               2  

[279 rows x 5 columns]